In [ ]:
import aduq.pyadm1 as adm1
import os
import numpy as np
data_path = 'aduq/pyadm1/data/'

# Loading data
feed = adm1.IO.load_dig_feed(os.path.join(data_path,"digester_influent.csv"))[:100] # limit to first 25 days to speed up
ini_state = adm1.IO.load_dig_state(os.path.join(data_path, "digester_initial.json"))
dig_info = adm1.IO.load_dig_info(os.path.join(data_path, "dig_info.json"))

param = adm1.IO.load_dig_param(os.path.join(data_path, "parameter.json"))

params_to_calib = ["K_S_ac", "k_m_ac"]

## ADM1 related classes

There are 5 main classes for ADM1 objects:
- DigesterInfluent: description of what goes in a digester. Saved as csv
- DigesterStates: observations of the digester. Saved as csv
- DigesterState: An observation of the digester at a given time. Saved as json
- DigesterInfo: Main info about the digester (volumes and temperatures, fixed). Saved as json
- DigesterParameter: description of the microbiology in the digester (what one calibrates). Saved as json

All objects are stored and displayed in human readable formats. However, they behave as np.ndarray! To access the pandas object, use the .to_pandas() method.
These objects can be saved using the save(path=) method.

In [ ]:
print(f"Digester info:\n{dig_info}\n")
print(f"Digester initial:\n{ini_state}\n")
print(f"Digester parameter:\n{param}\n")
print(f"Digester feed:\n{feed}\n")

## Basic operations

ADM1 model is called through run_adm1

In [ ]:
pred = adm1.run_adm1(param=param, influent_state=feed, initial_state=ini_state, digester_info=dig_info)

The derivative of the model is assessed using adm1_derivative function. This outputs an array of shape (p, t, d), where p is the number of parameter directions on which the derivative is computed, t the time and d the number of predictions (not counting time).

Note that the derivative computation is unstable, as it is impacted by the max_step argument used by the ordinary differential equation solver. Using low max_step value somewhat solves the problem, but results in an increased computation time. One should tweak the 'max_step' and 'rel_step' argument in adm1_derivative to assess the quality of the gradient estimation.

In [ ]:
adm1_der = adm1.adm1_derivative(
    param=param,
    params_to_der=params_to_calib,
    influent_state=feed,
    initial_state=ini_state,
    digester_info=dig_info,
    log_adm1=False
)

# One can assess the quality of the derivative in the following fashion
import numpy as np

param_mod = param.copy()
pred = adm1.run_adm1(param, feed, ini_state, dig_info, max_step= 0.5 / (60 * 24))
param_pd = param_mod.to_pandas()
perturb = np.random.normal(0,10 ** (-4), 2)
param_pd[params_to_calib] += perturb # param_mod is modified by side effect

pred_perturb = adm1.run_adm1(param_mod, feed, ini_state,  dig_info, max_step=0.5 / (60 * 24))
np.array((pred_perturb - pred))[:, 1:] / np.tensordot(perturb, adm1_der, (0, 0)) # These should be all 1.

## Sensitivity analysis

Routines for sensitivity analysis are implemented in module SA. A global sensitivity analysis based on Morris method or a local sensitivity analysis can be performed.

In [ ]:
adm1.SA.global_sensitivity(
    feed,
    ini_state,
    dig_info,
    r=10,
    n_lev=10,
) # Generates a lot of prints!

In [ ]:
adm1.SA.local_sensitivity(param, feed, ini_state, dig_info)

## Optimisation routines

In [ ]:
# To showcase the pyadm1 package, we generate noisy observations
from aduq.pyadm1._noise_data import noise_obs, noise_influent, noise_init_state
import matplotlib.pyplot as plt

obs = noise_obs(pred, noise_lev=0.1)
plt.plot(pred.to_pandas()['time'], pred.to_pandas()["S_ac"])
plt.plot(obs.to_pandas()['time'], obs.to_pandas()["S_ac"])
plt.show()
print(f"Prediction error: {adm1.adm1_err(pred, obs)}")

In [ ]:
init_param = param.copy()
param_pd = init_param.to_pandas()
param_pd[params_to_calib] = param_pd[params_to_calib] * np.random.uniform(
    0.4, 2.5, len(params_to_calib)
)  # 5% average change to every parameter

out = adm1.optim.optim_cma_adm1(
    init_param=init_param,
    obs=obs,
    params_eval=params_to_calib,
    chain_length=20,
    influent_state=feed,
    initial_state=ini_state,
    digester_info=dig_info,
    print_rec=4,
    radius_factor=0.5,
    radius_ini=.2
)
opti_param = out.opti_param

import matplotlib.pyplot as plt

plt.plot(out.hist_score)


### Bayesian calibration
__Iter prior procedure:__

The iter prior step is designed to speed up the early task of the Bayesian calibration procedure, using a variant of CMA-ES algorithm training only a diagonal covariance, with an exit criteria based on the variational inference score. It can be called using adm1_iter_prior_vi function from optim submodule. Note that the opti_param attribute of the resulting OptimResult object describes a tensorized gaussian parameter, not a gaussian parameter. 

In [ ]:
out = adm1.optim.adm1_iter_prior_vi(obs=obs, influent_state=feed, initial_state=ini_state, digester_info=dig_info,temperature=0.01, gen_per_step=60, chain_length=5, stop_tol=0.01, keep=30)

post_param = out.opti_param

# Conversion to parameter for GaussianMap
from aduq.proba.gauss import tgauss_to_gauss_param
post_param = tgauss_to_gauss_param(post_param)

__Gradient descent Bayes__

The bayesian calibration is performed using adm1_vi function from adm1.optim module.
This method is computationally intensive and should be performed using parallel computations on large virtual machines.
Note that the first steps are slow since building momentum is key.

The bayesian calibration uses the variational class defined in adm1.proba module.

In [ ]:
optim_res = adm1.optim.adm1_vi(
    obs=obs,
    influent_state=feed,
    initial_state=ini_state,
    digester_info=dig_info,
    ini_post=post_param,
    temperature=0.01,
    chain_length=3,
    index_train=adm1.distr_param_indexes,
    per_step=100,
    step_size=0.001,
    momentum=0.9,
    print_rec=1,
    parallel=True,
)

## UQ module

### Fisher's information

In [ ]:
fim_out = adm1.UQ.adm1_fim(opti_param, obs, feed, ini_state, dig_info, ['K_S_ac', 'k_m_ac'], silent=True)

In [ ]:
# One can access the p-value using adm1_fim_pval function
adm1.UQ.adm1_fim_pval(param, opti_param, cov = fim_out["cov"], inv_cov = fim_out["fisher"] )

In [ ]:
# And one can evaluate the uncertainty on the previsions using linear transfer of uncertainty and gaussian hypothesis

output_UQ = adm1.UQ.adm1_fim_pred(
    opti_predict=fim_out["opti_predict"],
    cov=fim_out["cov"],
    der_log_adm1=fim_out["der_log_adm1"],
    conf_lev=0.7,
)

low_quant = output_UQ["lower_quant"].to_pandas()
high_quant = output_UQ["upper_quant"].to_pandas()
plt.plot(low_quant["time"], low_quant["S_ac"], label="lower prediction quantile")
plt.plot(high_quant["time"], high_quant["S_ac"], label="higher prediction quantile")
plt.legend()
plt.ylabel("S_ac")
plt.show()

In [ ]:
beale_out = adm1.UQ.adm1_beale(20, conf_lev=.99, cov=fim_out['cov'], param=opti_param, params_eval=params_to_calib, obs=obs,influent_state=feed, initial_state=ini_state,digester_info=dig_info)

In [ ]:
plt.plot(beale_out["boundary"]["K_S_ac"], beale_out["boundary"]["k_m_ac"], ".")
plt.xlabel("K_S_ac")
plt.ylabel("k_m_ac")
plt.show()

In [ ]:
lin_boot_out = adm1.UQ.adm1_lin_bootstrap(10**4, obs=obs, opti_param=param, params_eval=params_to_calib, influent_state=feed, initial_state=ini_state, digester_info=dig_info)

In [ ]:
import pandas as pd
sample_boot = lin_boot_out["sample"]
sample_pd = pd.DataFrame(sample_boot, columns = list(adm1.IO.parameter_dict.keys()))
plt.plot(sample_pd["K_S_ac"], sample_pd["k_m_ac"], '.')